1. Download voxceleb files from this Drive: https://1drv.ms/u/s!AmYq9m5Um8eJjJwQdIIxEnX4_ELPBA?e=nEcboP 
2. create a function that takes in a wav file and returns a np.array
3. take Sinisa's samples and transform them to array
3. modify the create_databases() function to create dataframes (one df has a speaker_id and mfcc_id), the other has (mfcc_id, mfcc_chanel)

# VoxCeleb Data

In [18]:
import copy

import librosa
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from typing import Union
from torch.utils.data import Subset
import time
from tqdm import tqdm
from torchvision import transforms, models
from PIL import Image
from scipy.io.wavfile import read
import wave

from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset

In [3]:
import pickle

with open(r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\voxceleb\mfcc_channel_db_64000_16000_13_voxceleb_4s.pkl", 'rb') as f:
    mfcc_vox_channel = pickle.load(f)
    
with open(r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\voxceleb\speaker_mfcc_db_64000_16000_13_voxceleb_4s.pkl", 'rb') as f:
    speaker_vox_channel = pickle.load(f)


In [4]:
speaker_vox_channel

,speaker_id,mfcc_id
0,0,0
1,0,1
2,0,2
3,0,3
4,0,4
...,...,...
98539,1250,98539
98540,1250,98540
98541,1250,98541
98542,1250,98542


In [5]:
speaker_vox_channel.groupby('speaker_id').size().sort_values()

speaker_id
923       33
957       33
683       34
557       36
383       36
        ... 
19       867
934      935
1110     937
95      1230
985     1739
Length: 1251, dtype: int64

In [6]:
np.unique(speaker_vox_channel['speaker_id'])

array([0, 1, 2, ..., 1248, 1249, 1250], dtype=object)

In [7]:
mfcc_vox_channel

,channel_0,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,channel_9,channel_10,channel_11,channel_12
mfcc_id,,,,,,,,,,,,,
0,"[-187.24637, -149.53009, -151.10945, -153.8774...","[68.89113, 63.364716, 70.8166, 80.3503, 80.600...","[9.889414, 2.3946476, 2.5752912, -6.69186, -19...","[17.502277, 12.635658, 17.878536, 28.23352, 31...","[-13.575983, -14.448822, -13.248672, -19.11864...","[1.0645044, -2.6240473, -0.13270956, 2.2109358...","[-15.607198, -19.356922, -20.096199, -17.34228...","[-5.32699, -4.9090166, -3.775187, -11.470164, ...","[3.7866244, 3.7023747, 1.3626612, 3.5234811, 3...","[-14.94981, -12.564394, -13.128988, -9.048162,...","[-15.132621, -13.143087, -9.905177, -3.4710124...","[-3.8398762, -7.6827126, -6.4173346, -1.987698...","[-1.0505389, -5.991003, -10.116522, -15.312014..."
1,"[-198.294, -114.487236, -83.88371, -92.66598, ...","[42.30703, 28.736141, 37.711243, 75.62166, 92....","[-6.576665, -0.8904468, -17.23628, -32.981873,...","[47.598827, 54.884617, 52.340027, 45.405674, 4...","[-10.850039, -14.415745, -12.853505, -12.63353...","[1.8850632, -8.064954, -7.5922117, -1.0741322,...","[-5.038056, -17.551157, -25.905128, -25.009705...","[-12.357721, -20.44262, -33.268463, -41.169384...","[-4.636651, 4.1460056, 5.6635036, 2.4866378, -...","[-14.945326, -18.34693, -20.50335, -16.587328,...","[-10.585033, -7.709153, -10.281559, -6.2568793...","[-7.0390244, -13.208345, -14.768049, 0.5783107...","[-5.4636817, -2.6122441, -9.878437, -19.423641..."
2,"[-90.13957, -83.886154, -103.12938, -108.63772...","[79.63759, 71.0854, 63.44264, 70.60083, 82.133...","[-28.5051, -41.391636, -54.700657, -64.014854,...","[42.649612, 57.943947, 72.27799, 80.52837, 85....","[-34.345848, -40.075825, -46.01481, -46.01294,...","[-27.595322, -28.506674, -21.098446, -15.46166...","[-26.295082, -29.387112, -29.155825, -23.66281...","[-9.1275215, -6.4995055, -2.024608, -3.7854204...","[10.985041, 9.911264, 8.392603, 9.884792, 11.8...","[-9.172028, -8.2226925, -6.155219, -15.123125,...","[6.568225, 8.167038, 1.0410409, 4.462028, 12.9...","[9.280914, 9.741494, 8.6711445, 4.660267, -1.5...","[-8.119853, -10.304121, -18.021496, -14.767426..."
3,"[-97.715614, -95.80054, -137.73134, -160.00127...","[81.36412, 106.41818, 153.7273, 151.40231, 133...","[-47.480263, -54.040657, -36.57792, -6.042321,...","[3.355823, 7.716856, 15.211743, 16.140945, 13....","[-34.95617, -44.157013, -54.943058, -38.27945,...","[-0.74967146, -0.6681238, -3.9512663, -16.8522...","[-5.777954, -2.16335, 3.648534, -5.5802765, -8...","[-25.249668, -37.33124, -40.97127, -33.545906,...","[10.171267, 3.628562, -0.759184, 3.8890944, 11...","[10.010314, 8.174147, -10.33065, -21.193298, -...","[4.335509, 9.348421, 8.275001, -8.141726, -10....","[-8.289565, 1.4887252, 6.672163, -2.8742366, -...","[-21.479828, -16.664913, -14.358726, -13.74014..."
4,"[-176.24092, -117.82418, -104.70973, -99.88144...","[87.8996, 42.825783, -1.0114547, -15.791763, 2...","[37.66838, 57.93568, 61.87578, 57.950085, 37.3...","[43.399086, 23.913319, 10.752448, 13.6803055, ...","[1.8330331, 11.735545, 6.8757973, -0.73776174,...","[-4.03551, -9.102802, -15.085886, -19.169891, ...","[-11.921533, -14.827215, -22.301748, -25.92164...","[-8.102881, -8.4137335, -4.71406, -7.736088, -...","[-20.62529, -15.820948, -8.361216, -9.497086, ...","[-12.631435, -12.190712, -8.594565, -5.0830297...","[-16.870842, -15.69952, -9.652981, -8.370828, ...","[-26.852156, -15.576142, -3.9010525, -3.991034...","[-4.862768, -3.0103784, 2.9243217, 8.242981, 9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98539,"[-330.80063, -327.78192, -347.83435, -344.1253...","[143.96695, 138.84062, 146.81604, 171.92578, 1...","[5.2493825, 6.438201, 7.771367, 11.435141, -3....","[-18.89615, -6.2175303, 2.1640232, -6.4587803,...","[-39.425392, -31.90382, -26.53632, -30.591902,...","[22.205372, 18.779346, 13.847256, 4.271571, 5....","[-5.6084375, 0.5672497, 2.482668, -11.216707, ...","[-6.567702, -5.733662, -3.94

In [8]:
mfcc_vox_channel["channel_0"][0]

mfcc_id
0    [-187.24637, -149.53009, -151.10945, -153.8774...
0    [-329.12683, -312.28018, -347.7598, -380.52814...
Name: channel_0, dtype: object

In [143]:
def play_audio(waveform : torch.Tensor, sample_rate : int) -> None:
    """
    Plays audio given a waveform and sample rate
    
    """
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    if num_channels == 1:
        display(Audio(waveform[0], rate=sample_rate))
    elif num_channels == 2:
        display(Audio((waveform[0], waveform[1]), rate=sample_rate))
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")

In [10]:
def create_mfcc_databases(dataset, window_size, sample_rate, speaker_dict, number_spectral_coefficients):
    """
    Creates two databases from an audio dataset e.g. librispeech
    
    Inputs
    -----
    
    base_dataset : 
        A pytorch audio dataset like librispeech
    
    window_size : int
        window size to be used to consistently split waveforms

    sample_rate : int
        sample rate in kHz    
    
    speaker_dict : dict
        dictionary of {'speaker_id':encoded_id} pairs          

    number_spectral_coefficients : int
        number of spectral coefficients to keep for the MFCC calculation
        
    Returns
    -------
    speaker_mfcc_db : pd.DataFrame 
        df with shape (index_id, speaker_id, mfcc_id)
    
    mfcc_channel_db : pd.DataFrame
        df with shape (mfcc_id, channel_id)
    
    """
    try:
        speaker_mfcc_db = pd.read_pickle(f'speaker_mfcc_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')
        mfcc_channel_db = pd.read_pickle(f'mfcc_channel_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')
    
    except FileNotFoundError:
        speaker_mfcc_db = pd.DataFrame(columns=["speaker_id", "mfcc_id"])
        mfcc_channel_db = pd.DataFrame(columns=["mfcc_id"]+[f"channel_{i}" for i in range(number_spectral_coefficients)]).set_index('mfcc_id')

        idx = 0

        print('Creating a database. Hold tight...')
        for i in tqdm(range(dataset.__len__())):
            (waveform, sample_rate, transcript, speaker_id, chapter_id, utterance_id) = dataset.__getitem__(i)
            waveform_arr = waveform.numpy().flatten()
            splitted_waveform = split_waveform(waveform_arr, window_size) # split waveforms into consistent chunks

            for chunk in splitted_waveform: # transform waveforms into MFCC spectrograms
                mfccs = librosa.feature.mfcc(y=chunk.flatten(), n_mfcc=13, sr=sample_rate)

                speaker_mfcc_db.loc[idx, "speaker_id"] = speaker_dict[speaker_id]
                speaker_mfcc_db.loc[idx, "mfcc_id"] = idx

                for j in range(number_spectral_coefficients):
                    mfcc_channel_db.loc[idx, f"channel_{j}"] = 1
                    mfcc_channel_db.at[idx, f"channel_{j}"] = mfccs[j, :]
                
                idx += 1

        speaker_mfcc_db.to_pickle(f'speaker_mfcc_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')
        mfcc_channel_db.to_pickle(f'mfcc_channel_db_{window_size}_{sample_rate}_{number_spectral_coefficients}.pkl')

    return speaker_mfcc_db, mfcc_channel_db

In [11]:
def get_datasets(base_dataset, speaker_dict, window_size, sample_rate , train_pct , 
                 val_pct , test_pct , number_spectral_coefficients , cepstral_normalize , 
                 transform=None):
    """
    Wrapper around other functions to create datasets
    
    Inputs
    -----
    
    base_dataset : 
        A pytorch audio dataset like librispeech
    
    speaker_dict : dict
        dictionary of {'speaker_id':encoded_id} pairs          

    window_size : int
        window size to be used to consistently split waveforms

    sample_rate : int
        sample rate in kHz

    train_pct : float
        train percentage
    
    val_pct : float
        validation percentage

    test_pct : float
        Test percentage

    number_spectral_coefficients : int
        number of spectral coefficients to keep for the MFCC calculation
        
    normalize : bool
        whether to apply cepstral normalization
        
    transform : 
        a custom transformation to apply to the dataset
        
    Returns
    -------
    
    mfcc_dataset_train, mfcc_dataset_val, mfcc_dataset_test
    
    """
    speaker_mfcc_db, mfcc_channel_db = create_mfcc_databases(base_dataset, window_size=window_size, sample_rate=sample_rate, speaker_dict=speaker_dict, number_spectral_coefficients=number_spectral_coefficients)

    train_ids, val_ids, test_ids = split_train_test_val(speaker_mfcc_db, train_pct, val_pct, test_pct)
    
    speaker_mfcc_db_train = speaker_mfcc_db.loc[train_ids]
    speaker_mfcc_db_val = speaker_mfcc_db.loc[val_ids]
    speaker_mfcc_db_test = speaker_mfcc_db.loc[test_ids]
    
    if cepstral_normalize:
        cepstral_normalization = CepstralNormalization(number_spectral_coefficients)
        cepstral_normalization.fit(speaker_mfcc_db_train['mfcc_id'].values, mfcc_channel_db)
        normalized_mfcc_channel_db_train = cepstral_normalization.transform(speaker_mfcc_db_train['mfcc_id'].values, mfcc_channel_db)
        normalized_mfcc_channel_db_val = cepstral_normalization.transform(speaker_mfcc_db_val['mfcc_id'].values, mfcc_channel_db)
        normalized_mfcc_channel_db_test = cepstral_normalization.transform(speaker_mfcc_db_test['mfcc_id'].values, mfcc_channel_db)
        mfcc_channel_db = pd.concat([normalized_mfcc_channel_db_train, normalized_mfcc_channel_db_val, normalized_mfcc_channel_db_test])
        
    mfcc_channel_db_train = mfcc_channel_db.loc[speaker_mfcc_db_train['mfcc_id'].values]
    mfcc_channel_db_val = mfcc_channel_db.loc[speaker_mfcc_db_val['mfcc_id'].values]
    mfcc_channel_db_test = mfcc_channel_db.loc[speaker_mfcc_db_test['mfcc_id'].values]
    
    mfcc_dataset_train = MFCCData(speaker_mfcc_db_train, mfcc_channel_db_train, transform=transform)
    mfcc_dataset_val = MFCCData(speaker_mfcc_db_val, mfcc_channel_db_val, transform=transform)
    mfcc_dataset_test = MFCCData(speaker_mfcc_db_test, mfcc_channel_db_test, transform=transform)

    return mfcc_dataset_train, mfcc_dataset_val, mfcc_dataset_test

# Custom Celeb Data

In [149]:
mypath = r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\Celeb Data"
f = []
for (dirpath, dirnames, filenames) in os.walk(mypath):
    break

In [150]:
print("Folders")
dir_name_list = []
count = 0
for i, dirname in enumerate(dirnames):
    count += 1
    dir_name_list.append(dirname)
    print(i, ' ', dirname)

Folders
0   Alison Pill
1   Amy Poehler
2   Chadwick Boseman
3   Cillian Murphy
4   David Letterman
5   Sinisa


In [151]:

import os

def get_filepaths(directory):
    """
    This function will generate the file names in a directory 
    tree by walking the tree either top-down or bottom-up. For each 
    directory in the tree rooted at directory top (including top itself), 
    it yields a 3-tuple (dirpath, dirnames, filenames).
    """
    file_paths = []  # List which will store all of the full filepaths.

    # Walk the tree.
    for root, directories, files in os.walk(directory):
        for filename in files:
            # Join the two strings in order to form the full filepath.
            filepath = os.path.join(root, filename)
            file_paths.append(filepath)  # Add it to the list.

    return file_paths  # Self-explanatory.

# Run the above function and store its results in a variable.   
full_file_paths = get_filepaths(r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\Celeb Data")

In [152]:
name = full_file_paths[0]
content = name[113:-4]
content.split('\\')[1]

'A_Pill_1'

In [153]:
y_test, sr_test = librosa.load(name)
torch.tensor(y_test)

tensor([ 0.0118,  0.0233,  0.0209,  ..., -0.0082,  0.0060, -0.0233])

In [154]:
ifile = wave.open(name)
samples = ifile.getnframes()
audio = ifile.readframes(samples)
audio_as_np = np.frombuffer(audio, dtype="float32")
audio_as_np

array([ 9.2572618e-38,  2.5862228e-37,  3.8794394e-37, ...,
       -1.5056151e+35,  3.2033930e-37,            nan], dtype=float32)

In [155]:
ifile = wave.open(name)
samples = ifile.getnframes()
audio = ifile.readframes(samples)
audio_as_np = np.frombuffer(audio)
audio_as_np

array([ 9.79160597e-296,  1.56660589e-293,  1.41945291e-294, ...,
        2.81393696e-266, -1.70010689e+279, -5.61764107e+306])

In [156]:
filename = full_file_paths[0]
ifile = wave.open(filename)
samples = ifile.getnframes()
audio = ifile.readframes(samples)
audio_as_np = np.frombuffer(audio, dtype=float)
audio_as_np

array([ 9.79160597e-296,  1.56660589e-293,  1.41945291e-294, ...,
        2.81393696e-266, -1.70010689e+279, -5.61764107e+306])

In [171]:
name_list = []
unique_list = []
speaker_ID_list = []
for filename in full_file_paths:
    content = filename[113:-4]
    split_name = content.split('\\')[1]
    unique_list.append(split_name[0:4])
    name_list.append(split_name)    
    if split_name[0:4] == "A_Pi":
        speaker_ID = 10039
    elif split_name[0:4] == "A_Po":
        speaker_ID = 10046
    elif split_name[0:4] == "C_Bo":
        speaker_ID = 10140
    elif split_name[0:4] == "C_Mu":
        speaker_ID = 10212
    elif split_name[0:4] == "D_Le":
        speaker_ID = 10212
    elif split_name[0:4] == "Sini":
        speaker_ID = 1251
    speaker_ID_list.append(speaker_ID)   
    
    
    

In [172]:
speaker_ID_list

[10039,
 10039,
 10039,
 10039,
 10039,
 10039,
 10039,
 10039,
 10039,
 10039,
 10046,
 10046,
 10046,
 10046,
 10046,
 10046,
 10046,
 10046,
 10046,
 10046,
 10140,
 10140,
 10140,
 10140,
 10140,
 10140,
 10140,
 10140,
 10140,
 10140,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 10212,
 1251,
 1251,
 1251,
 1251,
 1251,
 1251,
 1251,
 1251,
 1251,
 1251]

In [168]:
np.unique(unique_list)

array(['A_Pi', 'A_Po', 'C_Bo', 'C_Mu', 'D_Le', 'Sini'], dtype='<U4')

In [ ]:

# import os

# def get_filepaths(directory):
#     """
#     This function will generate the file names in a directory 
#     tree by walking the tree either top-down or bottom-up. For each 
#     directory in the tree rooted at directory top (including top itself), 
#     it yields a 3-tuple (dirpath, dirnames, filenames).
#     """
#     file_paths = []  # List which will store all of the full filepaths.

#     # Walk the tree.
#     for root, directories, files in os.walk(directory):
#         for filename in files:
#             # Join the two strings in order to form the full filepath.
#             filepath = os.path.join(root, filename)
#             file_paths.append(filepath)  # Add it to the list.

#     return file_paths  # Self-explanatory.

# # Run the above function and store its results in a variable.   
# full_file_paths = get_filepaths(r"C:\Users\William Hazen\Documents\UofT\Term 1 (F)\MIE 1517\MIE 1517 Projects\audio-speaker-recognition\Celeb Data")

In [187]:
audio_as_np_list = []
sample_rate_list = []
name_list = []
mfccs_list = []
unique_list = []
speaker_ID_list = []

for filename in full_file_paths:
    content = filename[113:-4]
    split_name = content.split('\\')[1]
    unique_list.append(split_name[0:4])
    name_list.append(split_name)    
    if split_name[0:4] == "A_Pi":
        speaker_ID = 37
    elif split_name[0:4] == "A_Po":
        speaker_ID = 45 
    elif split_name[0:4] == "C_Bo":
        speaker_ID = 139
    elif split_name[0:4] == "C_Mu":
        speaker_ID = 165
    elif split_name[0:4] == "D_Le":
        speaker_ID = 211
    elif split_name[0:4] == "Sini":
        speaker_ID = 1251
    speaker_ID_list.append(speaker_ID)   
    
    y, sr = librosa.load(filename, sr=1600)
    audio_as_np_list.append(y)
    sample_rate_list.append(sr)
    mfccs = librosa.feature.mfcc(y=y, n_mfcc=13, sr=sr)
    mfccs_list.append(mfccs)

In [175]:
len(audio_as_np_list), len(sample_rate_list), len(name_list), len(speaker_ID_list), len(unique_list)

(60, 60, 60, 60, 60)

In [188]:
celeb_df = pd.DataFrame({
                        "Speaker Name": name_list,
                        "NP Waveforms": audio_as_np_list,
                        "Sample Rate": sample_rate_list,
                        "MFCC's": mfccs_list,
                        "speaker_ID_list": speaker_ID_list,
    
}) 

In [189]:
celeb_df

,Speaker Name,NP Waveforms,Sample Rate,MFCC's,speaker_ID_list
0,A_Pill_1,"[0.013411364, 0.023571905, 0.012701802, 0.0069...",1600,"[[-164.20244, -102.046425, -129.5111, -207.046...",37
1,A_Pill_10,"[0.0004919073, 9.4081144e-05, -0.0009698772, 0...",1600,"[[-320.23447, -247.98279, -175.2502, -121.1211...",37
2,A_Pill_2,"[-0.002918021, -0.0074602272, -0.0062654247, -...",1600,"[[-173.56845, -70.52583, -39.292255, -42.86666...",37
3,A_Pill_3,"[2.8710778e-07, -2.439551e-07, -7.316243e-07, ...",1600,"[[-190.6115, -78.34353, -64.12674, -114.326035...",37
4,A_Pill_4,"[0.13179499, 0.004738864, -0.011186932, 0.0098...",1600,"[[-52.51147, -47.4995, -74.43049, -73.38114, -...",37
5,A_Pill_5,"[1.2973961e-08, 5.1246523e-08, 6.123251e-08, -...",1600,"[[-557.2018, -160.70914, -66.946655, -87.05525...",37
6,A_Pill_6,"[0.024118165, -0.102908246, -0.036374018, -0.0...",1600,"[[-174.04979, -196.8654, -217.60712, -224.6349...",37
7,A_Pill_7,"[-5.2922707e-05, -0.00015098983, -7.670425e-05...",1600,"[[-282.86987, -196.08743, -185.2761, -160.4060...",37
8,A_Pill_8,"[-0.010201823, -0.026411835, -0.015610381, 0.0...",1600,"[[-164.984, -114.11463, -147.2735, -201.31615,...",37
9,A_Pill_9,"[-0.0003504373, 0.0026023318, -0.001371127, -0...",1600,"[[-342.54752, -303.15585, -266.97913, -214.921...",37


- id10039	Alison_Pill (39) -> (37)
- id10046	Amy_Poehler (47) -> (45)
- id10140	Chadwick_Boseman (141) -> (139)
- id10166	Cillian_Murphy	(167) -> (165)
- id10212	David_Letterman (213) -> (211)

In [200]:
sinisa_df = celeb_df[celeb_df["speaker_ID_list"] == 1251]

In [211]:
e_sinisa_df = sinisa_df.drop(["NP Waveforms", "Speaker Name", "Sample Rate"], axis=1)
e_sinisa_df["speaker_id"] = e_sinisa_df["speaker_ID_list"]
e_sinisa_df["mfcc"] = e_sinisa_df["MFCC's"]

In [212]:
e_sinisa_df

,MFCC's,speaker_ID_list,speaker_id,mfcc
50,"[[-150.0083, -113.192184, -132.91241, -170.337...",1251,1251,"[[-150.0083, -113.192184, -132.91241, -170.337..."
51,"[[-108.51038, -45.922688, -52.379158, -74.0102...",1251,1251,"[[-108.51038, -45.922688, -52.379158, -74.0102..."
52,"[[-307.35986, -246.52373, -229.26007, -198.597...",1251,1251,"[[-307.35986, -246.52373, -229.26007, -198.597..."
53,"[[-213.76352, -208.89221, -179.55733, -143.039...",1251,1251,"[[-213.76352, -208.89221, -179.55733, -143.039..."
54,"[[-171.19882, -93.35788, -73.2563, -98.05792, ...",1251,1251,"[[-171.19882, -93.35788, -73.2563, -98.05792, ..."
55,"[[-113.7628, -112.40339, -112.1716, -99.9618, ...",1251,1251,"[[-113.7628, -112.40339, -112.1716, -99.9618, ..."
56,"[[-102.9827, -115.125015, -194.65633, -184.110...",1251,1251,"[[-102.9827, -115.125015, -194.65633, -184.110..."
57,"[[-217.4453, -127.81827, -68.95807, -61.032726...",1251,1251,"[[-217.4453, -127.81827, -68.95807, -61.032726..."
58,"[[-671.4632, -671.4632, -671.4632, -663.3137, ...",1251,1251,"[[-671.4632, -671.4632, -671.4632, -663.3137, ..."
59,"[[-121.2038, -110.952126, -134.01118, -89.1948...",1251,1251,"[[-121.2038, -110.952126, -134.01118, -89.1948..."


In [215]:
e_sinisa_df = e_sinisa_df.drop(["MFCC's", "speaker_ID_list"], axis=1)

In [297]:
sinina_id = e_sinisa_df.drop(["mfcc"], axis=1)[0:7]
ext = [98544, 98545, 98546, 98547, 98548, 98549, 98550]

sinina_id["mfcc_id"] = [98544, 98545, 98546, 98547, 98548, 98549, 98550]
sinina_id = sinina_id.set_index(sinina_id.mfcc_id)

In [183]:

filename =  full_file_paths[0]

y, sr = librosa.load(filename)
mfccs = librosa.feature.mfcc(y=y, n_mfcc=13, sr=sr)
mfccs

array([[-315.24527   , -284.81128   , -283.25525   , ..., -129.46599   ,
        -115.147835  , -141.27301   ],
       [  99.57049   ,   85.89661   ,   76.73218   , ...,   54.32218   ,
          34.608192  ,    0.79439354],
       [   1.4887029 ,   -4.058543  ,  -11.636818  , ...,   -5.4683747 ,
          -3.0015993 ,    1.9328    ],
       ...,
       [  -9.977476  ,  -13.875602  ,  -10.922485  , ...,   -9.942442  ,
          -2.528006  ,    1.667707  ],
       [ -19.87944   ,  -21.716846  ,  -23.202488  , ...,    1.7382696 ,
           2.8970196 ,    8.32127   ],
       [   4.083488  ,   -1.2180246 ,  -10.841471  , ...,   -8.516981  ,
          -7.6570687 ,   -2.346367  ]], dtype=float32)

In [184]:

filename =  full_file_paths[0]

y, sr = librosa.load(filename)
mfccs = librosa.feature.mfcc(y=y, n_mfcc=13, sr=1600)
mfccs

array([[-270.45456  , -231.63275  , -225.08325  , ...,  -54.958595 ,
         -25.858362 ,  -25.359573 ],
       [  95.98001  ,   92.857765 ,   91.12718  , ...,   72.87709  ,
          58.849197 ,   29.573273 ],
       [   7.890584 ,   -1.6044667,   -2.4559772, ...,  -21.388416 ,
         -26.175922 ,  -29.343517 ],
       ...,
       [ -10.070546 ,   -9.74527  ,  -12.941551 , ...,  -17.052967 ,
         -15.707249 ,   -8.096355 ],
       [  13.228867 ,   13.543308 ,   10.469415 , ...,   22.815374 ,
          23.979864 ,   20.711506 ],
       [  11.09654  ,    8.446893 ,    5.0415225, ...,   -2.0751357,
          -4.020825 ,   -1.3664007]], dtype=float32)

In [292]:
temp_speaker_vox_channel = pd.DataFrame(speaker_vox_channel).copy()

In [293]:
temp_mfcc_vox_channel = pd.DataFrame(mfcc_vox_channel).copy()

In [294]:
index_val = temp_speaker_vox_channel.index.values.tolist() +  [98544, 98545, 98546, 98547, 98548, 98549, 98550]

In [300]:
temp_sp_vox_channel = pd.concat([temp_speaker_vox_channel, sinina_id])
temp_sp_vox_channel.set_index(temp_sp_vox_channel.mfcc_id)
temp_sp_vox_channel[-10:]

,speaker_id,mfcc_id
98541,1250,98541
98542,1250,98542
98543,1250,98543
98544,1251,98544
98545,1251,98545
98546,1251,98546
98547,1251,98547
98548,1251,98548
98549,1251,98549
98550,1251,98550


In [301]:
temp_mfcc_vox_channel

,channel_0,channel_1,channel_2,channel_3,channel_4,channel_5,channel_6,channel_7,channel_8,channel_9,channel_10,channel_11,channel_12
mfcc_id,,,,,,,,,,,,,
0,"[-187.24637, -149.53009, -151.10945, -153.8774...","[68.89113, 63.364716, 70.8166, 80.3503, 80.600...","[9.889414, 2.3946476, 2.5752912, -6.69186, -19...","[17.502277, 12.635658, 17.878536, 28.23352, 31...","[-13.575983, -14.448822, -13.248672, -19.11864...","[1.0645044, -2.6240473, -0.13270956, 2.2109358...","[-15.607198, -19.356922, -20.096199, -17.34228...","[-5.32699, -4.9090166, -3.775187, -11.470164, ...","[3.7866244, 3.7023747, 1.3626612, 3.5234811, 3...","[-14.94981, -12.564394, -13.128988, -9.048162,...","[-15.132621, -13.143087, -9.905177, -3.4710124...","[-3.8398762, -7.6827126, -6.4173346, -1.987698...","[-1.0505389, -5.991003, -10.116522, -15.312014..."
1,"[-198.294, -114.487236, -83.88371, -92.66598, ...","[42.30703, 28.736141, 37.711243, 75.62166, 92....","[-6.576665, -0.8904468, -17.23628, -32.981873,...","[47.598827, 54.884617, 52.340027, 45.405674, 4...","[-10.850039, -14.415745, -12.853505, -12.63353...","[1.8850632, -8.064954, -7.5922117, -1.0741322,...","[-5.038056, -17.551157, -25.905128, -25.009705...","[-12.357721, -20.44262, -33.268463, -41.169384...","[-4.636651, 4.1460056, 5.6635036, 2.4866378, -...","[-14.945326, -18.34693, -20.50335, -16.587328,...","[-10.585033, -7.709153, -10.281559, -6.2568793...","[-7.0390244, -13.208345, -14.768049, 0.5783107...","[-5.4636817, -2.6122441, -9.878437, -19.423641..."
2,"[-90.13957, -83.886154, -103.12938, -108.63772...","[79.63759, 71.0854, 63.44264, 70.60083, 82.133...","[-28.5051, -41.391636, -54.700657, -64.014854,...","[42.649612, 57.943947, 72.27799, 80.52837, 85....","[-34.345848, -40.075825, -46.01481, -46.01294,...","[-27.595322, -28.506674, -21.098446, -15.46166...","[-26.295082, -29.387112, -29.155825, -23.66281...","[-9.1275215, -6.4995055, -2.024608, -3.7854204...","[10.985041, 9.911264, 8.392603, 9.884792, 11.8...","[-9.172028, -8.2226925, -6.155219, -15.123125,...","[6.568225, 8.167038, 1.0410409, 4.462028, 12.9...","[9.280914, 9.741494, 8.6711445, 4.660267, -1.5...","[-8.119853, -10.304121, -18.021496, -14.767426..."
3,"[-97.715614, -95.80054, -137.73134, -160.00127...","[81.36412, 106.41818, 153.7273, 151.40231, 133...","[-47.480263, -54.040657, -36.57792, -6.042321,...","[3.355823, 7.716856, 15.211743, 16.140945, 13....","[-34.95617, -44.157013, -54.943058, -38.27945,...","[-0.74967146, -0.6681238, -3.9512663, -16.8522...","[-5.777954, -2.16335, 3.648534, -5.5802765, -8...","[-25.249668, -37.33124, -40.97127, -33.545906,...","[10.171267, 3.628562, -0.759184, 3.8890944, 11...","[10.010314, 8.174147, -10.33065, -21.193298, -...","[4.335509, 9.348421, 8.275001, -8.141726, -10....","[-8.289565, 1.4887252, 6.672163, -2.8742366, -...","[-21.479828, -16.664913, -14.358726, -13.74014..."
4,"[-176.24092, -117.82418, -104.70973, -99.88144...","[87.8996, 42.825783, -1.0114547, -15.791763, 2...","[37.66838, 57.93568, 61.87578, 57.950085, 37.3...","[43.399086, 23.913319, 10.752448, 13.6803055, ...","[1.8330331, 11.735545, 6.8757973, -0.73776174,...","[-4.03551, -9.102802, -15.085886, -19.169891, ...","[-11.921533, -14.827215, -22.301748, -25.92164...","[-8.102881, -8.4137335, -4.71406, -7.736088, -...","[-20.62529, -15.820948, -8.361216, -9.497086, ...","[-12.631435, -12.190712, -8.594565, -5.0830297...","[-16.870842, -15.69952, -9.652981, -8.370828, ...","[-26.852156, -15.576142, -3.9010525, -3.991034...","[-4.862768, -3.0103784, 2.9243217, 8.242981, 9..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
98539,"[-330.80063, -327.78192, -347.83435, -344.1253...","[143.96695, 138.84062, 146.81604, 171.92578, 1...","[5.2493825, 6.438201, 7.771367, 11.435141, -3....","[-18.89615, -6.2175303, 2.1640232, -6.4587803,...","[-39.425392, -31.90382, -26.53632, -30.591902,...","[22.205372, 18.779346, 13.847256, 4.271571, 5....","[-5.6084375, 0.5672497, 2.482668, -11.216707, ...","[-6.567702, -5.733662, -3.94

In [311]:
sin_mfcc_sp = e_sinisa_df.drop(["speaker_id"], axis=1)[0:7]


In [340]:
mf_list = sin_mfcc_sp["mfcc"].values
mf_list[0]

array([[-150.0083   , -113.192184 , -132.91241  , ..., -134.08347  ,
        -150.87366  , -200.21469  ],
       [  67.69481  ,   84.73309  ,  108.44603  , ...,   33.751568 ,
          58.52484  ,   91.18417  ],
       [ -38.24385  ,  -62.83754  ,  -81.26061  , ...,  -18.03008  ,
         -24.90793  ,  -31.537476 ],
       ...,
       [ -40.176918 ,  -46.965096 ,  -43.845337 , ...,  -29.000505 ,
         -22.192303 ,  -12.79657  ],
       [ -14.679676 ,  -18.292782 ,  -13.766966 , ...,   -1.0931615,
          -9.689476 ,  -13.624191 ],
       [  12.093235 ,   23.24943  ,    6.091381 , ...,  -20.705822 ,
         -11.486926 ,   -8.620178 ]], dtype=float32)

In [341]:
mf_list = sin_mfcc_sp["mfcc"].values

c=0
c1 = 0
channel_list = []
c_list = []
for count in mf_list:
    c_list.append(count)
    for channel in count:
       channel_list.append(channel)
        

In [345]:
len(c_list)

7

In [346]:
len(channel_list)

91

In [347]:
sin_mfcc_sp["mfcc_id"] = [98544, 98545, 98546, 98547, 98548, 98549, 98550]
sin_mfcc_sp = sin_mfcc_sp.set_index(sin_mfcc_sp.mfcc_id)
sin_mfcc_sp.drop(["mfcc_id", "mfcc"],axis=1)
sin_mfcc_sp["channel_0"] = channel_list


""
mfcc_id
98544
98545
98546
98547
98548
98549
98550


In [ ]:
sin_mfcc_sp.pivot_table()

In [ ]:
new_table = [["Row", "Column", "Data"]]
for line in table[1:]:
    for name, cell in zip(table[0], line)[1:]:
        new_line = [line[0], name, cell]
        new_table.append(new_line)

In [313]:
temp_mfcc_vox_channel = pd.concat([temp_mfcc_vox_channel, sin_mfcc_sp])

In [329]:
(temp_mfcc_vox_channel["channel_0"]).values

array([array([-187.24637 , -149.53009 , -151.10945 , -153.87744 , -157.39336 ,
              -167.64201 , -133.19125 , -110.70457 , -126.12043 , -128.65704 ,
              -101.416985,  -91.07796 , -108.27437 , -130.8411  , -116.01906 ,
               -94.87619 , -109.688965, -140.54437 , -157.03706 , -161.57188 ,
              -116.26887 ,  -68.43871 ,  -56.19124 ,  -77.701645,  -90.193   ,
               -99.70789 ,  -91.44558 ,  -91.830605, -116.76227 ,  -82.021774,
               -55.151524,  -67.72641 ,  -89.988785, -124.37212 , -179.39665 ,
              -200.58533 , -205.458   , -178.25829 , -119.096436,  -99.83642 ,
              -107.9164  , -117.80507 , -148.67375 , -171.24449 , -163.25299 ,
              -139.65823 , -103.064255,  -98.835884, -127.48778 , -163.86482 ,
              -138.19537 ,  -95.12578 , -100.05756 , -128.8513  , -123.48403 ,
              -115.790306, -120.04804 , -106.75805 ,  -98.03968 , -104.1702  ,
              -126.53275 , -165.43683 , -146.72174 ,

In [ ]:
# with open('E_Pickle.pkl', 'wb') as f:
#     pickle.dump(temp_sp_vox_channel, f)

# with open('S_Pickle.pkl', 'wb') as f:
#     pickle.dump(S, f)